In [3]:
from xcube.core.new import new_cube
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store
from xcube.core.store import new_data_writer

from xcube.core.dsio import open_dataset

In [4]:
data_store = new_data_store("s3", root="deep-esdl-output")

In [5]:
list(data_store.get_data_ids())



['black-sea-1x1024x1024.levels',
 'black-sea-1x1024x1024.zarr',
 'black-sea-256x128x128.zarr']

In [5]:
ds = data_store.open_data('black-sea-256x128x128.zarr')

## Create a copy of cube with different chunking

In [7]:
ds.VHM0.encoding

{'chunks': (256, 128, 128),
 'preferred_chunks': {'time': 256, 'lat': 128, 'lon': 128},
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 '_FillValue': nan,
 'dtype': dtype('float32')}

In [1]:
time_chunking = 1
lat_chunking = 1024
lon_chunking = 1024

In [ ]:
for variable in ds.data_vars:
    if variable != "crs":
        if ds[variable].encoding['chunks'] != (time_chunking, lat_chunking, lon_chunking):
            ds[variable].encoding['chunks'] = (time_chunking, lat_chunking, lon_chunking)
        print(variable)
        ds[variable] = ds[variable].chunk({'time': time_chunking, 'lat': lat_chunking, 'lon': lon_chunking})
        print(f'{variable} and {ds[variable].encoding["chunks"]}')
        #print(f'{variable} and {ds[variable].chunks}')

In [9]:
ds

,Array,Chunk
Bytes,43.57 GiB,4.00 MiB
Shape,"(731, 3000, 5333)","(1, 1024, 1024)"
Count,5 Graph Layers,13158 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,43.57 GiB,4.00 MiB
Shape,"(731, 3000, 5333)","(1, 1024, 1024)"
Count,5 Graph Layers,13158 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [10]:
output_id = 'black-sea-1x1024x1024.zarr'

In [11]:
data_store.write_data(ds, output_id)

'black-sea-1x1024x1024.zarr'

## Create multilevel dataset for faster visualisaiton in viewer:

For multilevel dataset use cube which contains 1 time stamp in the time chunk. For fast visualisation we only want to request information about one time stamp at a time. 

In [15]:
ml_dataset_id = 'black-sea-1x1024x1024.levels'
base_dataset_id = output_id # time chunking must be 1 of the base

In [13]:
ds = data_store.open_data(output_id)

In [14]:
ds

,Array,Chunk
Bytes,43.57 GiB,4.00 MiB
Shape,"(731, 3000, 5333)","(1, 1024, 1024)"
Count,2 Graph Layers,13158 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,43.57 GiB,4.00 MiB
Shape,"(731, 3000, 5333)","(1, 1024, 1024)"
Count,2 Graph Layers,13158 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [16]:
data_store.write_data(ds, 
                      ml_dataset_id, 
                      replace=True, 
                      base_dataset_id=base_dataset_id)

'black-sea-1x1024x1024.levels'